# ガバナンス状態を検証する
データガバナンス機能で作成したデータやGakunin RDMで付与したメタデータを検証、実験の再現性を確認するタスクです。

## 検証する
ガバナンスシートに設定された検証項目に従って、ガバナンスの状態を検証します。<br>
ガバナンスシートが未登録の場合は[こちら](./governance_sheet.ipynb)からガバナンスシートを登録してください。<br>
検証にはメタデータの登録が必要です。メタデータ未登録の場合は[こちら](./metadata.ipynb)からメタデータを登録してください。<br>
<br>
ガバナンスシートの「再現性のレベル」が「設定しない」以外で登録されている場合、GovernedRunのトークン入力が必要になります。<br>
※入力したトークンの有効性が確認された場合はリサーチフローに記憶されますので次回以降は入力の必要はありません。


In [ ]:
# 検証する
import os
import json
import traceback
from requests.exceptions import RequestException

import panel as pn
from IPython.display import display, clear_output
from IPython.core.display import Javascript

from library.utils import dg_web
from library.task_director import TaskDirector
from library.utils.html.button import create_button
from library.utils.config import message as msg_config
from library.utils.widgets import MessageBox, Button, Alert
from library.utils.storage_provider import grdm
from library.utils.error import UnusableVault, RemoteFileNotExist
from library.utils.input import get_grdm_token, get_project_id, get_goveredrun_token

notebook_name = 'validate.ipynb'

class Validator(TaskDirector):

    def __init__(self, working_path:str) -> None:
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    def load_grdm_token(self):
        """トークンを取得する"""
        token = ""
        try:
            token = get_grdm_token()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(str(e))
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(message)
            self.log.error(str(e))
        # エラーでも処理を中断させない
        # フォーム表示まで実行させる
        return token

    def load_goveredrun_token(self):
        """トークンを取得する"""
        token = ""
        try:
            token = get_goveredrun_token()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(str(e))
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(message)
            self.log.error(str(e))
        # エラーでも処理を中断させない
        # フォーム表示まで実行させる
        return token

    def get_data(self, token, base_url, project_id, remote_path):
        """既存データを取得する"""
        data = {}
        try:
            data = grdm.download_json_file(
                token=token, base_url=base_url,
                project_id=project_id, remote_path=remote_path
            )
        except RemoteFileNotExist:
            # 既存データが無かった場合
            pass
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(message)
            self.log.error(str(e))
        # エラーでも処理を中断させない
        # フォーム表示まで実行させる
        return data

    def alert_missing_file(self, path, msg):
        alert = Alert.warning(msg)
        if not os.path.isfile(path):
            return [alert]
        button_width = 500
        obj = create_button(
            url=f'{path}?init_nb=true',
            target='_blank',
            msg=msg_config.get('dg_web', 'access_register'),
            button_width=f'{button_width}px'
        )
        button = pn.pane.HTML(obj, width=button_width)
        return [alert, button]

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # パラメータ取得
            self.grdm_token = self.load_grdm_token()
            self.project_id = get_project_id()
            self.govrun_token = None
            self.govsheet_path = dg_web.GOVSHEET_PATH
            self.metadata_path = dg_web.METADATA_PATH
            self.base_url = grdm.API_V2_BASE_URL
            clear_output()

            # ガバナンスシートとメタデータの確認
            govsheet = self.get_data(
                    token=self.grdm_token, base_url=self.base_url,
                    project_id=self.project_id, remote_path=self.govsheet_path
                )
            metadata = self.get_data(
                    token=self.grdm_token, base_url=self.base_url,
                    project_id=self.project_id, remote_path=self.metadata_path
                )
            if not govsheet:
                path = './governance_sheet.ipynb'
                msg = msg_config.get('dg_web', 'missing_govsheet')
                self._form_box.extend(self.alert_missing_file(path, msg))
            elif not metadata:
                path = './metadata.ipynb'
                msg = msg_config.get('dg_web', 'missing_metadata')
                self._form_box.extend(self.alert_missing_file(path, msg))
            else:
                # 再現性検証の準備
                if dg_web.need_govrun_token(govsheet, metadata):
                    self.govrun_token = self.load_goveredrun_token()

            # ボタンの表示
            if len(self._msg_output.objects) < 1 and len(self._form_box.objects) < 1:
                self.submit_button_title = msg_config.get('dg_web', 'validate')
                self.submit_button = Button(width=500)
                self.submit_button.set_looks_init(self.submit_button_title)
                self.submit_button.on_click(self.submit)
                self._form_box.append(self.submit_button)

        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    @TaskDirector.callback_form('検証する')
    def submit(self, event):
        self.submit_button.set_looks_processing(msg_config.get('dg_web', 'validating'))
        try:
            result = dg_web.validate(scheme=dg_web.SCHEME, domain=dg_web.DOMAIN, grdm_token=self.grdm_token, project_id=self.project_id, govrun_token=self.govrun_token)
        except RequestException as e:
            error_summary = traceback.format_exception_only(type(e), e)[0].rstrip('\\n')
            error_msg = msg_config.get('DEFAULT', 'connection_error') + "\n" + error_summary
            self.log.error(error_msg)
            self._msg_output.update_error(error_msg)
            self.submit_button.set_looks_init(self.submit_button_title)
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.log.error(message)
            self._msg_output.update_error(message)
            self.submit_button.set_looks_init(self.submit_button_title)
            return
        self._form_box.clear()
        title = '# ' + msg_config.get('dg_web', 'validation_result')
        self._form_box.append(pn.pane.Markdown(title))
        result = json.dumps(result, ensure_ascii=False, indent=4)
        styles = {
            'background': '#f5f5f5',
            'border': '1px solid black',
            'padding': '10px'
        }
        self._form_box.append(pn.pane.Str(result, styles=styles))

Validator(os.path.abspath('__file__')).generateFormScetion()

## 検証結果を確認する
検証結果の一覧を表示します。<br>
再現性検証が実施されている場合、検証実施直後には検証結果が得られない可能性があります。しばらく時間を置いてから再度確認してください。<br>

In [ ]:
# 検証結果を確認する

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

script_file_name = "validate"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()